In [2]:
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
     
def pre_processData(newsgroups_train):
    tokenizer = RegexpTokenizer(r'\w+')
    for i in range(len(newsgroups_train)):
        newsgroups_train[i] = newsgroups_train[i].lower()
        #newsgroups_train[i] = tokenizer.tokenize(newsgroups_train[i])
    #newsgroups_train = [[token for token in doc if not token.isdigit()] for doc in newsgroups_train]
    newsgroups_train = [doc.split(' ') for doc in newsgroups_train]
    return newsgroups_train
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from collections import defaultdict
import gensim
from InitKmeans import InitKmeans
import numpy as np
import pandas as pd
from sklearn.mixture import GaussianMixture
import sys  
import json
if __name__ == "__main__":
    newsgroups_train = list()
    news_labels = {}
    docID_username = {}
    
    i = 0
    file_path="D:/part_news4_biterm.txt"
    with open(file_path) as fp:
        lines = fp.read().split("\n")
        for line in lines:
            if line:
                text = json.loads(line)["bitermText"].strip()
                label = json.loads(line)["clusterNo"]
                newsgroups_train.append(text)
                news_labels[i] = label
                i+=1
    fp.close()
    
    corpus = pre_processData(newsgroups_train)

D:\anacoda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
doc_biterm = {}
for docID in range(len(corpus)):
    words = corpus[docID]
    duplicate_biterm = []
    doc_biterm[docID] = {}
    for biterm in words:
        hash_biterm = set(biterm.split(','))
        if hash_biterm not in duplicate_biterm:
            duplicate_biterm.append(hash_biterm)
            doc_biterm[docID][biterm] = 1
        else:
            if biterm in doc_biterm[docID].keys():
                doc_biterm[docID][biterm] += 1
            else:
                doc_biterm[docID][biterm] = 1
doc_words = {}
i_biterm = 0
for docID in range(len(corpus)):
    doc_words[docID] = {}
    words = corpus[docID]
    for biterm in words:
        for word in biterm.split(','):
            if word not in doc_words[docID].keys():
                doc_words[docID][word] = 0
            doc_words[docID][word] += 1

In [9]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.5,5.5,0.5):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  97
iter:  0  ,docID  2000
len(Topics)  121
iter:  0  ,docID  3000
len(Topics)  132
iter:  0  ,docID  4000
len(Topics)  141
iter:  0  ,docID  5000
len(Topics)  144
iter:  0  ,docID  6000
len(Topics)  149
iter:  0  ,docID  7000
len(Topics)  151


KeyboardInterrupt: 

In [ ]:
print(alpha_NMI)
print(alpha_topics)

In [5]:
print(alpha_NMI)
print(alpha_topics)

{0.5: 0.8680239094660858, 1.0: 0.8706042742918579, 1.5: 0.8757233687872634, 2.0: 0.8461538635964255}
{0.5: 308, 1.0: 539, 1.5: 411, 2.0: 372}


In [10]:
alpha_NMI = {}
alpha_topics = {}
for alpha in np.arange(0.1,1.1,0.1):
    alpha = np.around(alpha,decimals=1)
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    #alpha = 1
    beta = 0.02
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(len(corpus)):
            if docID % 1000 == 0:
                print("iter: ",iter," ,docID ",docID)
                print("len(Topics) ",len(Topics))
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   

        from sklearn.metrics.cluster import normalized_mutual_info_score
        nmi_sample = []
        nmi_result = []
        for key, value in news_labels.items():
                nmi_sample.append(value)
                nmi_result.append(docID_assign_z[key])
        print("alpha ", alpha, "Topics ",len(Topics), " iter ",iter," ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
        alpha_NMI[alpha] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
        alpha_topics[alpha] = len(Topics)

iter:  0  ,docID  0
len(Topics)  0
iter:  0  ,docID  1000
len(Topics)  95
iter:  0  ,docID  2000
len(Topics)  115
iter:  0  ,docID  3000
len(Topics)  125
iter:  0  ,docID  4000
len(Topics)  130
iter:  0  ,docID  5000
len(Topics)  131
iter:  0  ,docID  6000
len(Topics)  132
iter:  0  ,docID  7000
len(Topics)  134
iter:  0  ,docID  8000
len(Topics)  136
iter:  0  ,docID  9000
len(Topics)  146
iter:  0  ,docID  10000
len(Topics)  150
iter:  0  ,docID  11000
len(Topics)  154
iter:  0  ,docID  12000
len(Topics)  156
iter:  0  ,docID  13000
len(Topics)  159
iter:  0  ,docID  14000
len(Topics)  161
iter:  0  ,docID  15000
len(Topics)  164
iter:  0  ,docID  16000
len(Topics)  165
iter:  0  ,docID  17000
len(Topics)  165
iter:  0  ,docID  18000
len(Topics)  168
iter:  0  ,docID  19000
len(Topics)  168
iter:  0  ,docID  20000
len(Topics)  172
iter:  0  ,docID  21000
len(Topics)  174
iter:  0  ,docID  22000
len(Topics)  175
iter:  0  ,docID  23000
len(Topics)  176
iter:  0  ,docID  24000
len(Topi

iter:  6  ,docID  9000
len(Topics)  259
iter:  6  ,docID  10000
len(Topics)  261
iter:  6  ,docID  11000
len(Topics)  261
iter:  6  ,docID  12000
len(Topics)  262
iter:  6  ,docID  13000
len(Topics)  262
iter:  6  ,docID  14000
len(Topics)  262
iter:  6  ,docID  15000
len(Topics)  262
iter:  6  ,docID  16000
len(Topics)  261
iter:  6  ,docID  17000
len(Topics)  262
iter:  6  ,docID  18000
len(Topics)  262
iter:  6  ,docID  19000
len(Topics)  262
iter:  6  ,docID  20000
len(Topics)  262
iter:  6  ,docID  21000
len(Topics)  264
iter:  6  ,docID  22000
len(Topics)  264
iter:  6  ,docID  23000
len(Topics)  264
iter:  6  ,docID  24000
len(Topics)  264
iter:  6  ,docID  25000
len(Topics)  266
iter:  6  ,docID  26000
len(Topics)  267
iter:  6  ,docID  27000
len(Topics)  267
iter:  6  ,docID  28000
len(Topics)  268
iter:  6  ,docID  29000
len(Topics)  268
iter:  6  ,docID  30000
len(Topics)  268
alpha  0.1 Topics  268  iter  6   0.8572711290024694
iter:  7  ,docID  0
len(Topics)  268
iter:  7 

iter:  2  ,docID  18000
len(Topics)  226
iter:  2  ,docID  19000
len(Topics)  226
iter:  2  ,docID  20000
len(Topics)  226
iter:  2  ,docID  21000
len(Topics)  226
iter:  2  ,docID  22000
len(Topics)  227
iter:  2  ,docID  23000
len(Topics)  226
iter:  2  ,docID  24000
len(Topics)  228
iter:  2  ,docID  25000
len(Topics)  227
iter:  2  ,docID  26000
len(Topics)  229
iter:  2  ,docID  27000
len(Topics)  229
iter:  2  ,docID  28000
len(Topics)  229
iter:  2  ,docID  29000
len(Topics)  229
iter:  2  ,docID  30000
len(Topics)  229
alpha  0.2 Topics  229  iter  2   0.8434212460488048
iter:  3  ,docID  0
len(Topics)  229
iter:  3  ,docID  1000
len(Topics)  229
iter:  3  ,docID  2000
len(Topics)  231
iter:  3  ,docID  3000
len(Topics)  232
iter:  3  ,docID  4000
len(Topics)  233
iter:  3  ,docID  5000
len(Topics)  232
iter:  3  ,docID  6000
len(Topics)  235
iter:  3  ,docID  7000
len(Topics)  236
iter:  3  ,docID  8000
len(Topics)  238
iter:  3  ,docID  9000
len(Topics)  238
iter:  3  ,docID 

iter:  8  ,docID  26000
len(Topics)  290
iter:  8  ,docID  27000
len(Topics)  291
iter:  8  ,docID  28000
len(Topics)  291
iter:  8  ,docID  29000
len(Topics)  293
iter:  8  ,docID  30000
len(Topics)  293
alpha  0.2 Topics  293  iter  8   0.8610843023385276
iter:  9  ,docID  0
len(Topics)  293
iter:  9  ,docID  1000
len(Topics)  292
iter:  9  ,docID  2000
len(Topics)  290
iter:  9  ,docID  3000
len(Topics)  290
iter:  9  ,docID  4000
len(Topics)  289
iter:  9  ,docID  5000
len(Topics)  290
iter:  9  ,docID  6000
len(Topics)  290
iter:  9  ,docID  7000
len(Topics)  289
iter:  9  ,docID  8000
len(Topics)  289
iter:  9  ,docID  9000
len(Topics)  289
iter:  9  ,docID  10000
len(Topics)  289
iter:  9  ,docID  11000
len(Topics)  289
iter:  9  ,docID  12000
len(Topics)  289
iter:  9  ,docID  13000
len(Topics)  289
iter:  9  ,docID  14000
len(Topics)  288
iter:  9  ,docID  15000
len(Topics)  287
iter:  9  ,docID  16000
len(Topics)  287
iter:  9  ,docID  17000
len(Topics)  285
iter:  9  ,docID 

iter:  5  ,docID  2000
len(Topics)  303
iter:  5  ,docID  3000
len(Topics)  303
iter:  5  ,docID  4000
len(Topics)  306
iter:  5  ,docID  5000
len(Topics)  307
iter:  5  ,docID  6000
len(Topics)  310
iter:  5  ,docID  7000
len(Topics)  311
iter:  5  ,docID  8000
len(Topics)  313
iter:  5  ,docID  9000
len(Topics)  313
iter:  5  ,docID  10000
len(Topics)  313
iter:  5  ,docID  11000
len(Topics)  315
iter:  5  ,docID  12000
len(Topics)  312
iter:  5  ,docID  13000
len(Topics)  313
iter:  5  ,docID  14000
len(Topics)  314
iter:  5  ,docID  15000
len(Topics)  315
iter:  5  ,docID  16000
len(Topics)  315
iter:  5  ,docID  17000
len(Topics)  319
iter:  5  ,docID  18000
len(Topics)  319
iter:  5  ,docID  19000
len(Topics)  320
iter:  5  ,docID  20000
len(Topics)  320
iter:  5  ,docID  21000
len(Topics)  322
iter:  5  ,docID  22000
len(Topics)  320
iter:  5  ,docID  23000
len(Topics)  320
iter:  5  ,docID  24000
len(Topics)  319
iter:  5  ,docID  25000
len(Topics)  319
iter:  5  ,docID  26000


...

iter:  8  ,docID  30000
len(Topics)  394
alpha  0.7 Topics  394  iter  8   0.8661652694153419
iter:  9  ,docID  0
len(Topics)  394
iter:  9  ,docID  1000
len(Topics)  396
iter:  9  ,docID  2000
len(Topics)  394
iter:  9  ,docID  3000
len(Topics)  394
iter:  9  ,docID  4000
len(Topics)  396
iter:  9  ,docID  5000
len(Topics)  397
iter:  9  ,docID  6000
len(Topics)  394
iter:  9  ,docID  7000
len(Topics)  392
iter:  9  ,docID  8000
len(Topics)  393
iter:  9  ,docID  9000
len(Topics)  390
iter:  9  ,docID  10000
len(Topics)  389
iter:  9  ,docID  11000
len(Topics)  389
iter:  9  ,docID  12000
len(Topics)  387
iter:  9  ,docID  13000
len(Topics)  388
iter:  9  ,docID  14000
len(Topics)  386
iter:  9  ,docID  15000
len(Topics)  385
iter:  9  ,docID  16000
len(Topics)  385
iter:  9  ,docID  17000
len(Topics)  384
iter:  9  ,docID  18000
len(Topics)  383
iter:  9  ,docID  19000
len(Topics)  384
iter:  9  ,docID  20000
len(Topics)  383
iter:  9  ,docID  21000
len(Topics)  383
iter:  9  ,docID 

iter:  5  ,docID  6000
len(Topics)  365
iter:  5  ,docID  7000
len(Topics)  364
iter:  5  ,docID  8000
len(Topics)  363
iter:  5  ,docID  9000
len(Topics)  364
iter:  5  ,docID  10000
len(Topics)  365
iter:  5  ,docID  11000
len(Topics)  367
iter:  5  ,docID  12000
len(Topics)  370
iter:  5  ,docID  13000
len(Topics)  369
iter:  5  ,docID  14000
len(Topics)  368
iter:  5  ,docID  15000
len(Topics)  373
iter:  5  ,docID  16000
len(Topics)  373
iter:  5  ,docID  17000
len(Topics)  372
iter:  5  ,docID  18000
len(Topics)  373
iter:  5  ,docID  19000
len(Topics)  372
iter:  5  ,docID  20000
len(Topics)  370
iter:  5  ,docID  21000
len(Topics)  371
iter:  5  ,docID  22000
len(Topics)  370
iter:  5  ,docID  23000
len(Topics)  370
iter:  5  ,docID  24000
len(Topics)  372
iter:  5  ,docID  25000
len(Topics)  371
iter:  5  ,docID  26000
len(Topics)  370
iter:  5  ,docID  27000
len(Topics)  369
iter:  5  ,docID  28000
len(Topics)  368
iter:  5  ,docID  29000
len(Topics)  370
iter:  5  ,docID  30

iter:  1  ,docID  15000
len(Topics)  325
iter:  1  ,docID  16000
len(Topics)  329
iter:  1  ,docID  17000
len(Topics)  333
iter:  1  ,docID  18000
len(Topics)  332
iter:  1  ,docID  19000
len(Topics)  334
iter:  1  ,docID  20000
len(Topics)  335
iter:  1  ,docID  21000
len(Topics)  340
iter:  1  ,docID  22000
len(Topics)  349
iter:  1  ,docID  23000
len(Topics)  349
iter:  1  ,docID  24000
len(Topics)  353
iter:  1  ,docID  25000
len(Topics)  353
iter:  1  ,docID  26000
len(Topics)  354
iter:  1  ,docID  27000
len(Topics)  356
iter:  1  ,docID  28000
len(Topics)  355
iter:  1  ,docID  29000
len(Topics)  356
iter:  1  ,docID  30000
len(Topics)  352
alpha  0.9 Topics  352  iter  1   0.8405573505797204
iter:  2  ,docID  0
len(Topics)  352
iter:  2  ,docID  1000
len(Topics)  357
iter:  2  ,docID  2000
len(Topics)  360
iter:  2  ,docID  3000
len(Topics)  364
iter:  2  ,docID  4000
len(Topics)  366
iter:  2  ,docID  5000
len(Topics)  366
iter:  2  ,docID  6000
len(Topics)  365
iter:  2  ,doc

iter:  7  ,docID  23000
len(Topics)  406
iter:  7  ,docID  24000
len(Topics)  406
iter:  7  ,docID  25000
len(Topics)  404
iter:  7  ,docID  26000
len(Topics)  409
iter:  7  ,docID  27000
len(Topics)  411
iter:  7  ,docID  28000
len(Topics)  410
iter:  7  ,docID  29000
len(Topics)  409
iter:  7  ,docID  30000
len(Topics)  410
alpha  0.9 Topics  410  iter  7   0.8659703330417178
iter:  8  ,docID  0
len(Topics)  410
iter:  8  ,docID  1000
len(Topics)  409
iter:  8  ,docID  2000
len(Topics)  408
iter:  8  ,docID  3000
len(Topics)  408
iter:  8  ,docID  4000
len(Topics)  408
iter:  8  ,docID  5000
len(Topics)  409
iter:  8  ,docID  6000
len(Topics)  411
iter:  8  ,docID  7000
len(Topics)  410
iter:  8  ,docID  8000
len(Topics)  408
iter:  8  ,docID  9000
len(Topics)  408
iter:  8  ,docID  10000
len(Topics)  406
iter:  8  ,docID  11000
len(Topics)  405
iter:  8  ,docID  12000
len(Topics)  405
iter:  8  ,docID  13000
len(Topics)  405
iter:  8  ,docID  14000
len(Topics)  405
iter:  8  ,docID 

iter:  4  ,docID  1000
len(Topics)  383
iter:  4  ,docID  2000
len(Topics)  384
iter:  4  ,docID  3000
len(Topics)  386
iter:  4  ,docID  4000
len(Topics)  387
iter:  4  ,docID  5000
len(Topics)  385
iter:  4  ,docID  6000
len(Topics)  385
iter:  4  ,docID  7000
len(Topics)  386
iter:  4  ,docID  8000
len(Topics)  386
iter:  4  ,docID  9000
len(Topics)  384
iter:  4  ,docID  10000
len(Topics)  384
iter:  4  ,docID  11000
len(Topics)  385
iter:  4  ,docID  12000
len(Topics)  386
iter:  4  ,docID  13000
len(Topics)  386
iter:  4  ,docID  14000
len(Topics)  384
iter:  4  ,docID  15000
len(Topics)  384
iter:  4  ,docID  16000
len(Topics)  383
iter:  4  ,docID  17000
len(Topics)  383
iter:  4  ,docID  18000
len(Topics)  383
iter:  4  ,docID  19000
len(Topics)  382
iter:  4  ,docID  20000
len(Topics)  383
iter:  4  ,docID  21000
len(Topics)  384
iter:  4  ,docID  22000
len(Topics)  383
iter:  4  ,docID  23000
len(Topics)  384
iter:  4  ,docID  24000
len(Topics)  385
iter:  4  ,docID  25000
l

In [11]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.8667865634503141, 0.8656853471524877, 0.8665974423500961, 0.8709520495334826, 0.8640038818629493, 0.8655290262237872, 0.870667958105711, 0.8614436874163404, 0.8694058094984684, 0.8686112631279349])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [12]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([266, 281, 321, 341, 337, 336, 374, 374, 395, 393])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [7]:
print(alpha_NMI.values())
print(alpha_NMI.keys())

dict_values([0.8630841853405375, 0.8590541089822986, 0.8687674321235832, 0.8644798941318023, 0.861674271985366, 0.867613577256499, 0.8694736783205159, 0.8718119277938999, 0.8706175047674019, 0.8701634637462661])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [8]:
print(alpha_topics.values())
print(alpha_topics.keys())

dict_values([231, 268, 277, 303, 330, 302, 334, 364, 381, 363])
dict_keys([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])


In [4]:
import timeit
total_batch = 16
NMI_batch = {}
start = 0
end = 0
topic_batch = {}
time_iteration_list = []
for batch_i in range(1,17):
    if batch_i == 16:
        end = int(len(corpus))
    else:
        end = int(len(corpus) / total_batch) * batch_i
    
    docID_assign_z = {}
    m_z = {}
    n_z = {}
    n_w = {}
    n_b = {}
    Topics = []
    V = set()
    D = set()
    alpha = 0.3
    beta = 0.02
    NMI_batch[batch_i] = 0
    topic_batch[batch_i] = 0
    
    time_start = timeit.default_timer()
    total_iter = 10
    for iter in range(total_iter):
        for docID in range(start,end):
            words = corpus[docID]
            D.discard(docID)
            if docID in docID_assign_z.keys():
                before_k = docID_assign_z[docID]
                m_z[before_k].discard(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        n_z[before_k][word] -= 1
                        n_w[before_k] -=1
            else:
                before_k = -1
            if len(D) == 0 and len(V) == 0:
                choose_k = 0
                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)
            else:
                log_pro = []
                for k in Topics:
                    pro_k = len(m_z[k])
                    if pro_k != 0:
                        i = 0
                        for biterm in words:
                            each_word = biterm.strip().split(',')
                            for word in each_word:
                                if word not in n_z[k].keys():
                                    n_z[k][word] = 0
                            for j in range(doc_biterm[docID][biterm]):
                                pro_k *= (n_z[k][each_word[0]] + n_z[k][each_word[-1]] + beta + j) / ( n_w[k] + len(V)*beta + i) 
                                i += 1

                    if pro_k == 0:
                        pro_k = sys.float_info.min
                    log_pro.append(pro_k)

                pro_new_k = alpha*(len(D))
                i = 0
                for biterm in words:
                    for j in range(doc_biterm[docID][biterm]):
                        pro_new_k *= ( beta + j) / ( len(V)*beta + i) 
                        i += 1

                if pro_new_k == 0:
                    pro_new_k = sys.float_info.min        
                log_pro.append(pro_new_k)

                sum_pro=sum(log_pro)

                normalized_posterior = [i/sum_pro for i in log_pro]    
                select_k = None
                if iter == (total_iter - 1):
                    select_k = normalized_posterior.index(max(normalized_posterior))

                else:
                    select_k = np.random.choice( (len(Topics)+1) , 1, p=normalized_posterior)[0]  

                if select_k == len(Topics):
                    choose_k = np.max(Topics) + 1
                else:
                    choose_k = Topics[select_k]

                D.add(docID)
                docID_assign_z[docID] = choose_k
                if choose_k not in m_z.keys():
                    m_z[choose_k] = set()
                m_z[choose_k].add(docID)
                for biterm in words:
                    for word in biterm.split(','):
                        if choose_k not in n_w.keys():
                            n_w[choose_k] = 0
                        if choose_k not in n_z.keys():
                            n_z[choose_k] = {}
                        if word not in n_z[choose_k].keys():
                            n_z[choose_k][word] = 0
                        n_z[choose_k][word] += 1
                        n_w[choose_k] += 1
                        V.add(word)
                if choose_k not in Topics:
                    Topics.append(choose_k)

            count_k = []
            for k in Topics:
                if k in m_z.keys() and len(m_z[k]) == 0:
                    m_z.pop(k, None)
                    n_z.pop(k, None)
                    n_w.pop(k, None)
                    count_k.append(k)
            for k in count_k:
                Topics.remove(k)   
        if iter == 9:
            from sklearn.metrics.cluster import normalized_mutual_info_score
            nmi_sample = []
            nmi_result = []
            for key, value in news_labels.items():
                if key < end and key >= start:
                    nmi_sample.append(value)
                    nmi_result.append(docID_assign_z[key])
            NMI_batch[batch_i] = normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result))
            topic_batch[batch_i] = len(Topics)
            print("start ",start, " end ",end, "topics: ",len(Topics),"truth: ",len(np.unique(nmi_sample)), " NMI: ",normalized_mutual_info_score(np.array(nmi_sample), np.array(nmi_result)))
#     if batch_i != 1:
#         start = int(len(corpus) / total_batch) * (batch_i -1) + 1
    time_stop = timeit.default_timer()
    time_iteration_list.append(time_stop-time_start)
    print(time_iteration_list)

start  0  end  1895 topics:  125 truth:  89  NMI:  0.8410839966127501
[226.21218378253968]
start  0  end  3790 topics:  150 truth:  98  NMI:  0.834194791775418
[226.21218378253968, 578.0604549883734]
start  0  end  5685 topics:  166 truth:  104  NMI:  0.8357588712834243
[226.21218378253968, 578.0604549883734, 859.6052154228396]
start  0  end  7580 topics:  178 truth:  108  NMI:  0.8317386819103495
[226.21218378253968, 578.0604549883734, 859.6052154228396, 1087.1243275125453]
start  0  end  9475 topics:  202 truth:  175  NMI:  0.8343894508861612
[226.21218378253968, 578.0604549883734, 859.6052154228396, 1087.1243275125453, 1405.792722277292]
start  0  end  11370 topics:  227 truth:  194  NMI:  0.847137619069315
[226.21218378253968, 578.0604549883734, 859.6052154228396, 1087.1243275125453, 1405.792722277292, 1749.0887730012873]
start  0  end  13265 topics:  225 truth:  202  NMI:  0.8591345644310551
[226.21218378253968, 578.0604549883734, 859.6052154228396, 1087.1243275125453, 1405.792722

In [5]:
print(topic_batch.keys())
print(topic_batch.values())

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
dict_values([125, 150, 166, 178, 202, 227, 225, 240, 245, 263, 266, 279, 289, 295, 304, 344])
